# COVID-19 World Vaccination Progress

This dataset provied a big variety of data concerning the vaccination progress across different countries in the world. The complete description of the columns can be found below:
* Country- this is the country for which the vaccination information is provided;
* Country ISO Code - ISO code for the country;
* Date - date for the data entry; for some of the dates we have only the daily vaccinations, for others, only the (cumulative) total;
* Total number of vaccinations - this is the absolute number of total immunizations in the country;
* Total number of people vaccinated - a person, depending on the immunization scheme, will receive one or more (typically 2) vaccines; at a certain moment, the number of vaccination might be larger than the number of people;
* Total number of people fully vaccinated - this is the number of people that received the entire set of immunization according to the immunization scheme (typically 2); at a certain moment in time, there might be a certain number of people that received one vaccine and another number (smaller) of people that received all vaccines in the scheme;
* Daily vaccinations (raw) - for a certain data entry, the number of vaccination for that date/country
* Daily vaccinations - for a certain data entry, the number of vaccination for that date/country;
* Total vaccinations per hundred - ratio (in percent) between vaccination number and total population up to the date in the country;
* Total number of people vaccinated per hundred - ratio (in percent) between population immunized and total population up to the date in the country;
* Total number of people fully vaccinated per hundred - ratio (in percent) between population fully immunized and total population up to the date in the country;
* Daily vaccinations per million - ratio (in ppm) between vaccination number and total population for the current date in the country;
* Vaccines used in the country - total number of vaccines used in the country (up to date);
* Source name - source of the information (national authority, international organization, local organization etc.);
* Source website - website of the source of information;

Our task is to expand this dataset with further information about each country's socio-economical situation and enable queries that clarify their progress in the vaccination race.

Below we tried to solve some of the dataset's problems. There exist many NaN values,so we forward-filled those and set to 0 those that cannot be filled. Although this process produces a believable dataset without missing values, it may produce some overestimation in some occasions. Another solution is to set all those values to 0, which may produse an underestimation. 

We will validate the results that we take with other official sources and assess the impact that our intervention has in the results from out desired task. The final preprocessing of the dataset will be decided at a later point, also accounting for the extra information that will be added from different sources.

# New section

In [1]:
!pip install google-cloud-bigquery
!pip install pandas
!pip install numpy
!pip install pyarrow

You should consider upgrading via the 'c:\users\lefteris\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\lefteris\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\lefteris\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\lefteris\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import os

df = pd.read_csv("./data/country_vaccinations.csv")
df.head()

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15666 entries, 0 to 15665
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   country                              15666 non-null  object 
 1   iso_code                             15666 non-null  object 
 2   date                                 15666 non-null  object 
 3   total_vaccinations                   9437 non-null   float64
 4   people_vaccinated                    8754 non-null   float64
 5   people_fully_vaccinated              6502 non-null   float64
 6   daily_vaccinations_raw               7928 non-null   float64
 7   daily_vaccinations                   15465 non-null  float64
 8   total_vaccinations_per_hundred       9437 non-null   float64
 9   people_vaccinated_per_hundred        8754 non-null   float64
 10  people_fully_vaccinated_per_hundred  6502 non-null   float64
 11  daily_vaccinations_per_milli

In [4]:
#fill NA values with forward fill
df_filled = df.groupby('country').fillna(method='ffill')

#the rest NA values can be set to 0
df_filled.fillna(0, inplace=True)
df_filled["country"] = df["country"]

#reorder columns for ease of use
cols_to_order = ['country', 'iso_code', 'date', 'vaccines']
new_columns = cols_to_order + (df_filled.columns.drop(cols_to_order).tolist())
df_filled = df_filled[new_columns]
df_filled.head()

,country,iso_code,date,vaccines,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,source_name,source_website
0,Afghanistan,AFG,2021-02-22,Oxford/AstraZeneca,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
1,Afghanistan,AFG,2021-02-23,Oxford/AstraZeneca,0.0,0.0,0.0,0.0,1367.0,0.0,0.0,0.0,35.0,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
2,Afghanistan,AFG,2021-02-24,Oxford/AstraZeneca,0.0,0.0,0.0,0.0,1367.0,0.0,0.0,0.0,35.0,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
3,Afghanistan,AFG,2021-02-25,Oxford/AstraZeneca,0.0,0.0,0.0,0.0,1367.0,0.0,0.0,0.0,35.0,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
4,Afghanistan,AFG,2021-02-26,Oxford/AstraZeneca,0.0,0.0,0.0,0.0,1367.0,0.0,0.0,0.0,35.0,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...


In [5]:
df_filled[df_filled['vaccines'].str.contains('Sputnik')].head()

,country,iso_code,date,vaccines,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,source_name,source_website
60,Albania,ALB,2021-01-10,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ...",0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,Ministry of Health,https://twitter.com/gmanastirliu/status/139037...
61,Albania,ALB,2021-01-11,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ...",0.0,0.0,0.0,0.0,64.0,0.00,0.00,0.0,22.0,Ministry of Health,https://twitter.com/gmanastirliu/status/139037...
62,Albania,ALB,2021-01-12,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ...",128.0,128.0,0.0,0.0,64.0,0.00,0.00,0.0,22.0,Ministry of Health,https://twitter.com/gmanastirliu/status/139037...
63,Albania,ALB,2021-01-13,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ...",188.0,188.0,0.0,60.0,63.0,0.01,0.01,0.0,22.0,Ministry of Health,https://twitter.com/gmanastirliu/status/139037...
64,Albania,ALB,2021-01-14,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ...",266.0,266.0,0.0,78.0,66.0,0.01,0.01,0.0,23.0,Ministry of Health,https://twitter.com/gmanastirliu/status/139037...


In [6]:
df_filled[df_filled['country'].str.contains('Algeria')].head()

,country,iso_code,date,vaccines,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,source_name,source_website
177,Algeria,DZA,2021-01-29,Sputnik V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Ministry of Health,https://www.echoroukonline.com/%d9%84%d9%82%d8...
178,Algeria,DZA,2021-01-30,Sputnik V,30.0,0.0,0.0,30.0,30.0,0.0,0.0,0.0,1.0,Ministry of Health,https://www.echoroukonline.com/%d9%84%d9%82%d8...
179,Algeria,DZA,2021-01-31,Sputnik V,30.0,0.0,0.0,30.0,1889.0,0.0,0.0,0.0,43.0,Ministry of Health,https://www.echoroukonline.com/%d9%84%d9%82%d8...
180,Algeria,DZA,2021-02-01,Sputnik V,30.0,0.0,0.0,30.0,2509.0,0.0,0.0,0.0,57.0,Ministry of Health,https://www.echoroukonline.com/%d9%84%d9%82%d8...
181,Algeria,DZA,2021-02-02,Sputnik V,30.0,0.0,0.0,30.0,2819.0,0.0,0.0,0.0,64.0,Ministry of Health,https://www.echoroukonline.com/%d9%84%d9%82%d8...


---------------------------------------------
#### Which Vaccines are used per Country?
---------------------------------------------

In [7]:
vaccines = df_filled.vaccines.unique()
for v in vaccines:
    countries = df_filled.loc[df_filled.vaccines==v, 'country'].values
    print(f"Vaccines: {v}: \nCountries: {(np.unique(countries))}\n")

Vaccines: Oxford/AstraZeneca: 
Countries: ['Afghanistan' 'Angola' 'Anguilla' 'Antigua and Barbuda' 'Bahamas'
 'Bangladesh' 'Barbados' 'Belize' 'Bhutan' 'Botswana' 'Brunei'
 'Cape Verde' 'Comoros' "Cote d'Ivoire" 'Democratic Republic of Congo'
 'Djibouti' 'Dominica' 'Eswatini' 'Ethiopia' 'Falkland Islands' 'Fiji'
 'Gambia' 'Georgia' 'Ghana' 'Grenada' 'Guyana' 'Jamaica' 'Kosovo'
 'Lesotho' 'Malawi' 'Mali' 'Mauritius' 'Montserrat' 'Myanmar' 'Nauru'
 'Nigeria' 'Papua New Guinea' 'Saint Helena' 'Saint Kitts and Nevis'
 'Saint Lucia' 'Saint Vincent and the Grenadines' 'Samoa'
 'Sao Tome and Principe' 'Sierra Leone' 'Solomon Islands' 'South Sudan'
 'Sudan' 'Suriname' 'Taiwan' 'Timor' 'Togo' 'Tonga' 'Trinidad and Tobago'
 'Uganda' 'Uzbekistan' 'Vietnam' 'Zambia']

Vaccines: Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, Sputnik V: 
Countries: ['Albania' 'Bosnia and Herzegovina']

Vaccines: Sputnik V: 
Countries: ['Algeria' 'Armenia' 'Belarus' 'Guinea' 'Kazakhstan' 'Paraguay' 'Syria'
 'Venezuela

In [8]:
df.describe()

,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million
count,9.437000e+03,8.754000e+03,6.502000e+03,7.928000e+03,1.546500e+04,9437.000000,8754.000000,6502.000000,15465.000000
mean,5.250013e+06,3.302754e+06,1.672178e+06,1.371965e+05,7.948445e+04,16.130694,11.678865,5.945094,3007.500873
std,2.163269e+07,1.233716e+07,7.225403e+06,5.294140e+05,3.641596e+05,23.841769,15.622004,10.306655,4693.064582
min,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,6.306400e+04,5.403675e+04,2.461400e+04,3.071750e+03,9.100000e+02,1.400000,1.312500,0.660000,356.000000
50%,4.462850e+05,3.398010e+05,1.818105e+05,1.613000e+04,6.495000e+03,6.650000,5.130000,2.480000,1504.000000
75%,2.003211e+06,1.387596e+06,7.228470e+05,6.386600e+04,3.003600e+04,20.850000,15.510000,6.760000,4020.000000
max,2.977340e+08,1.494623e+08,1.089266e+08,1.160100e+07,7.205286e+06,215.710000,112.750000,102.950000,118759.000000


In [9]:
df_filled.describe()

,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million
count,1.566600e+04,1.566600e+04,1.566600e+04,1.566600e+04,1.566600e+04,15666.000000,15666.000000,15666.000000,15666.000000
mean,3.493592e+06,2.010736e+06,7.450212e+05,7.695068e+04,7.846563e+04,13.430149,9.481672,3.518440,2969.430869
std,1.724201e+07,9.499262e+06,4.740490e+06,3.888407e+05,3.619261e+05,22.756454,14.879813,8.760336,4675.074478
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,1.561200e+04,8.449000e+03,0.000000e+00,0.000000e+00,8.500000e+02,0.520000,0.300000,0.000000,332.000000
50%,1.441160e+05,9.531200e+04,3.178000e+03,2.350500e+03,6.108500e+03,3.995000,2.840000,0.110000,1469.000000
75%,1.027415e+06,7.427955e+05,1.644885e+05,2.593525e+04,2.929900e+04,16.510000,11.780000,3.050000,3998.250000
max,2.977340e+08,1.494623e+08,1.089266e+08,1.160100e+07,7.205286e+06,215.710000,112.750000,102.950000,118759.000000


# Other Useful Datasets 
Based on the above dataset an idea is to associate the epidemic characteristics with data relating to:


* Health Nutririon & Population Statistics
 https://www.kaggle.com/theworldbank/health-nutrition-and-population-statistics
 
 These data include population dynamics, nutrition, reproductive health, health financing, medical resources and usage, immunization, infectious diseases, HIV/AIDS, DALY, population projections and lending. 

*   World bank data
https://www.kaggle.com/theworldbank/world-bank-intl-education

  This dataset combines key education statistics from a variety of sources to provide a look at global literacy, spending, and access.

The main questions that arose during the analysis of the data, are the tracing of possible events that led to a different evolution of vaccinations in different countries. We consider the most important factors to be related to socio-economic factors and thus, we will try to extract knowledge using the above datasets in parallel. A review of the health data before the outbreak of the pandemic and the economic situation of each country may significantly explain not only the evolution of the vaccinations but also  the general course of the pandemic by country.

* Aggregation ανά μήνα

* Μετατροπή dataset σε γραμμες χώρες, στήλες δεδομένα

* World bank data ξεκαθάρισμα και μετατροπή σε γραμμή χώρα, στήλη δεδομένα + (un data)

* Εννοποίηση των dataset 

* Δημιουργία triple stores

* Queries



In [10]:
df.head()

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...


In [11]:
#df1 = df
df1 = df.copy() # Use this instead Copy in New Memory Position

df1['date'] = pd.to_datetime(df1['date'])

#fill total vaccinations NA values with forward fill 
df1["total_vaccinations"] = df1.groupby('country')["total_vaccinations"].fillna(method='ffill')

#fill dailyVaccinations NA values with zeroes
df1["daily_vaccinations"] = df1.groupby('country')["daily_vaccinations"].fillna(0)
df1.head(10)

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,0.0,0.00,0.00,NaN,NaN,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
1,Afghanistan,AFG,2021-02-23,0.0,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
2,Afghanistan,AFG,2021-02-24,0.0,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
3,Afghanistan,AFG,2021-02-25,0.0,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
4,Afghanistan,AFG,2021-02-26,0.0,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
5,Afghanistan,AFG,2021-02-27,0.0,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
6,Afghanistan,AFG,2021-02-28,8200.0,8200.0,NaN,NaN,1367.0,0.02,0.02,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
7,Afghanistan,AFG,2021-03-01,8200.0,NaN,NaN,NaN,1580.0,NaN,NaN,NaN,41.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
8,Afghanistan,AFG,2021-03-02,8200.0,NaN,NaN,NaN,1794.0,NaN,NaN,NaN,46.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
9,Afghanistan,AFG,2021-03-03,8200.0,NaN,NaN,NaN,2008.0,NaN,NaN,NaN,52.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...


In [12]:
drop_list = ["people_vaccinated", "people_fully_vaccinated", "daily_vaccinations_raw",
                                          "people_vaccinated_per_hundred","people_fully_vaccinated_per_hundred",
                                          "source_name" , "source_website", "total_vaccinations_per_hundred",	"daily_vaccinations_per_million"]
df1.drop(drop_list, axis=1, inplace=True)
df1.head()

,country,iso_code,date,total_vaccinations,daily_vaccinations,vaccines
0,Afghanistan,AFG,2021-02-22,0.0,0.0,Oxford/AstraZeneca
1,Afghanistan,AFG,2021-02-23,0.0,1367.0,Oxford/AstraZeneca
2,Afghanistan,AFG,2021-02-24,0.0,1367.0,Oxford/AstraZeneca
3,Afghanistan,AFG,2021-02-25,0.0,1367.0,Oxford/AstraZeneca
4,Afghanistan,AFG,2021-02-26,0.0,1367.0,Oxford/AstraZeneca


In [13]:
import datetime
df1["month"] = pd.DatetimeIndex(df1['date']).month
df2 = pd.DataFrame()
df2["total_vaccinations"] = df1.groupby(["country", "month"]).max()["total_vaccinations"]
df2["daily_vaccinations"] = df1.groupby(["country", "month"]).mean()["daily_vaccinations"]
df2["vaccines"] = df1.groupby(["country", "month"]).first()["vaccines"]
df2["iso_code"] = df1.groupby(["country", "month"]).first()["iso_code"]
df2.reset_index(inplace=True)
df2.head(10)

,country,month,total_vaccinations,daily_vaccinations,vaccines,iso_code
0,Afghanistan,2,8200.0,1171.714286,Oxford/AstraZeneca,AFG
1,Afghanistan,3,54000.0,2770.774194,Oxford/AstraZeneca,AFG
2,Afghanistan,4,240000.0,5727.272727,Oxford/AstraZeneca,AFG
3,Albania,1,549.0,33.681818,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ...",ALB
4,Albania,2,10135.0,339.428571,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ...",ALB
5,Albania,3,138846.0,2535.161290,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ...",ALB
6,Albania,4,476903.0,11300.766667,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ...",ALB
7,Albania,5,572226.0,15874.666667,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ...",ALB
8,Algeria,1,30.0,639.666667,Sputnik V,DZA
9,Algeria,2,75000.0,3534.263158,Sputnik V,DZA


In [14]:
vaccs = []
for s in df2["vaccines"].unique():
  for vacc in s.split(", "):
    if vacc not in vaccs:
      vaccs.append(vacc)

for vacc in vaccs:
  df2[vacc] = 0 
  df2.loc[df2["vaccines"].str.contains(vacc), vacc] = 1

In [15]:
df3 = df2.copy()
df3["january_total_vaccinations"] = 0
df3.loc[df3["month"] == 1, "january_total_vaccinations"] = df3.loc[df3["month"] == 1, "total_vaccinations"]
df3["february_total_vaccinations"] = 0
df3.loc[df3["month"] == 2, "february_total_vaccinations"] = df3.loc[df3["month"] == 2, "total_vaccinations"]
df3["march_total_vaccinations"] = 0
df3.loc[df3["month"] == 3, "march_total_vaccinations"] = df3.loc[df3["month"] == 3, "total_vaccinations"]
df3["april_total_vaccinations"] = 0
df3.loc[df3["month"] == 4, "april_total_vaccinations"] = df3.loc[df3["month"] == 4, "total_vaccinations"]
df3["may_total_vaccinations"] = 0
df3.loc[df3["month"] == 5, "may_total_vaccinations"] = df3.loc[df3["month"] == 5, "total_vaccinations"]

df3["january_daily_vaccinations"] = 0
df3.loc[df3["month"] == 1, "january_daily_vaccinations"] = df3.loc[df3["month"] == 1, "daily_vaccinations"]
df3["february_daily_vaccinations"] = 0
df3.loc[df3["month"] == 2, "february_daily_vaccinations"] = df3.loc[df3["month"] == 2, "daily_vaccinations"]
df3["march_daily_vaccinations"] = 0
df3.loc[df3["month"] == 3, "march_daily_vaccinations"] = df3.loc[df3["month"] == 3, "daily_vaccinations"]
df3["april_daily_vaccinations"] = 0
df3.loc[df3["month"] == 4, "april_daily_vaccinations"] = df3.loc[df3["month"] == 4, "daily_vaccinations"]
df3["may_daily_vaccinations"] = 0
df3.loc[df3["month"] == 5, "may_daily_vaccinations"] = df3.loc[df3["month"] == 5, "daily_vaccinations"]

In [16]:
df3 = df3.groupby("country").max().reset_index()
df3.head(10)

,country,month,total_vaccinations,daily_vaccinations,vaccines,iso_code,Oxford/AstraZeneca,Pfizer/BioNTech,Sinovac,Sputnik V,...,january_total_vaccinations,february_total_vaccinations,march_total_vaccinations,april_total_vaccinations,may_total_vaccinations,january_daily_vaccinations,february_daily_vaccinations,march_daily_vaccinations,april_daily_vaccinations,may_daily_vaccinations
0,Afghanistan,4,240000.0,5727.272727,Oxford/AstraZeneca,AFG,1,0,0,0,...,0.0,8200.0,54000.0,240000.0,0.0,0.000000,1171.714286,2770.774194,5727.272727,0.000000
1,Albania,5,572226.0,15874.666667,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ...",ALB,1,1,1,1,...,549.0,10135.0,138846.0,476903.0,572226.0,33.681818,339.428571,2535.161290,11300.766667,15874.666667
2,Algeria,2,75000.0,3534.263158,Sputnik V,DZA,0,0,0,1,...,30.0,75000.0,0.0,0.0,0.0,639.666667,3534.263158,0.000000,0.000000,0.000000
3,Andorra,5,28881.0,579.733333,"Oxford/AstraZeneca, Pfizer/BioNTech",AND,1,1,0,0,...,576.0,2526.0,9288.0,28503.0,28881.0,56.571429,61.071429,276.741935,579.733333,89.000000
4,Angola,4,456349.0,12529.869565,Oxford/AstraZeneca,AGO,1,0,0,0,...,0.0,0.0,130750.0,456349.0,0.0,0.000000,0.000000,3987.709677,12529.869565,0.000000
5,Anguilla,4,6898.0,170.600000,Oxford/AstraZeneca,AIA,1,0,0,0,...,0.0,3929.0,5348.0,6898.0,0.0,0.000000,170.600000,55.096774,47.454545,0.000000
6,Antigua and Barbuda,4,29754.0,826.153846,Oxford/AstraZeneca,ATG,1,0,0,0,...,0.0,0.0,26836.0,29754.0,0.0,0.000000,826.153846,599.354839,111.590909,0.000000
7,Argentina,12,8591061.0,126539.200000,"Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V",ARG,1,0,0,1,...,366384.0,1025003.0,3891932.0,7865213.0,8591061.0,10902.516129,18709.785714,89058.064516,126539.200000,122909.000000
8,Armenia,3,565.0,0.000000,Sputnik V,ARM,0,0,0,1,...,0.0,0.0,565.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
9,Aruba,5,79553.0,1646.433333,Pfizer/BioNTech,ABW,0,1,0,0,...,0.0,0.0,25766.0,76865.0,79553.0,0.000000,0.000000,852.000000,1646.433333,716.000000


In [17]:
columns_to_drop = ["month", "vaccines", "total_vaccinations", "daily_vaccinations"]
df3.drop(columns_to_drop, axis=1, inplace=True)
df3.head(10)

,country,iso_code,Oxford/AstraZeneca,Pfizer/BioNTech,Sinovac,Sputnik V,Sinopharm/Beijing,Johnson&Johnson,Moderna,Sinopharm/Wuhan,...,january_total_vaccinations,february_total_vaccinations,march_total_vaccinations,april_total_vaccinations,may_total_vaccinations,january_daily_vaccinations,february_daily_vaccinations,march_daily_vaccinations,april_daily_vaccinations,may_daily_vaccinations
0,Afghanistan,AFG,1,0,0,0,0,0,0,0,...,0.0,8200.0,54000.0,240000.0,0.0,0.000000,1171.714286,2770.774194,5727.272727,0.000000
1,Albania,ALB,1,1,1,1,0,0,0,0,...,549.0,10135.0,138846.0,476903.0,572226.0,33.681818,339.428571,2535.161290,11300.766667,15874.666667
2,Algeria,DZA,0,0,0,1,0,0,0,0,...,30.0,75000.0,0.0,0.0,0.0,639.666667,3534.263158,0.000000,0.000000,0.000000
3,Andorra,AND,1,1,0,0,0,0,0,0,...,576.0,2526.0,9288.0,28503.0,28881.0,56.571429,61.071429,276.741935,579.733333,89.000000
4,Angola,AGO,1,0,0,0,0,0,0,0,...,0.0,0.0,130750.0,456349.0,0.0,0.000000,0.000000,3987.709677,12529.869565,0.000000
5,Anguilla,AIA,1,0,0,0,0,0,0,0,...,0.0,3929.0,5348.0,6898.0,0.0,0.000000,170.600000,55.096774,47.454545,0.000000
6,Antigua and Barbuda,ATG,1,0,0,0,0,0,0,0,...,0.0,0.0,26836.0,29754.0,0.0,0.000000,826.153846,599.354839,111.590909,0.000000
7,Argentina,ARG,1,0,0,1,1,0,0,0,...,366384.0,1025003.0,3891932.0,7865213.0,8591061.0,10902.516129,18709.785714,89058.064516,126539.200000,122909.000000
8,Armenia,ARM,0,0,0,1,0,0,0,0,...,0.0,0.0,565.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
9,Aruba,ABW,0,1,0,0,0,0,0,0,...,0.0,0.0,25766.0,76865.0,79553.0,0.000000,0.000000,852.000000,1646.433333,716.000000


Below we add the info about religiosity and GDP per Capita to our constructed dataset.

In [18]:
df_religion = pd.read_csv("data/religion_vs_GDP_per_Capita.csv")
df_religion.replace({"country": {"Macedonia": "North Macedonia",\
                                "Czech Republic": "Czechia",\
                                "Democratic Republic of the Congo": "Democratic Republic of Congo"}}, inplace = True)

df4 = pd.merge(df3, df_religion, on = "country")

df4.drop(["Unnamed: 0", "US$"], axis = 1, inplace = True)

df_population = pd.read_csv("data/population_by_country_2020.csv")
df_population.drop(["Yearly Change", "Net Change", "Density (P/Km²)", "Land Area (Km²)", \
                    "Migrants (net)", "Fert. Rate", "Med. Age", "World Share"], axis = 1, inplace = True)
df_population = df_population.loc[df_population["Urban Pop %"] != "N.A."]
df_population.rename(columns = {"Country (or dependency)" : "country", "Population (2020)": "population", \
                               "Urban Pop %": "urban_population_percent"}, inplace = True)

df_population.replace({"country": {"Czech Republic (Czechia)": "Czechia", \
                                  "DR Congo": "Democratic Republic of Congo"}}, inplace = True)

df5 = pd.merge(df4, df_population, on = "country")

df5["population"] = pd.to_numeric(df5["population"])

df5["january_total_vaccinations"] = df5["january_total_vaccinations"] / df5["population"] * 100
df5["february_total_vaccinations"] = df5["february_total_vaccinations"] / df5["population"] * 100
df5["march_total_vaccinations"] = df5["march_total_vaccinations"] / df5["population"] * 100
df5["april_total_vaccinations"] = df5["april_total_vaccinations"] / df5["population"] * 100
df5["may_total_vaccinations"] = df5["may_total_vaccinations"] / df5["population"] * 100

df5["january_daily_vaccinations"] = df5["january_daily_vaccinations"] / df5["population"] * 100
df5["february_daily_vaccinations"] = df5["february_daily_vaccinations"] / df5["population"] * 100
df5["march_daily_vaccinations"] = df5["march_daily_vaccinations"] / df5["population"] * 100
df5["april_daily_vaccinations"] = df5["april_daily_vaccinations"] / df5["population"] * 100
df5["may_daily_vaccinations"] = df5["may_daily_vaccinations"] / df5["population"] * 100

df5["urban_population_percent"] = df5["urban_population_percent"].str.split(" ", expand = True)
df5["urban_population_percent"] = pd.to_numeric(df5["urban_population_percent"])

df5["religiousity%"] = pd.to_numeric(df5["religiousity%"])
#religiousity%
df5.shape

(129, 26)

-----------------------------------
## World Bank Education Database 
-----------------------------------

In [19]:
from google.cloud import bigquery 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="data/path/to/json"
client = bigquery.Client()

dataset_ref = client.dataset( 'world_bank_intl_education', project = 'bigquery-public-data')
dataset = client.get_dataset(dataset_ref)

tables = list(client.list_tables(dataset))

# Show all tables in the Dataset 
for table in tables:
    print(table.table_id)
    

# Create a reference to the Internationa-Education Table 
table_ref = dataset_ref.table('international_education')
table_Educ = client.get_table(table_ref)

#Info for the Dataset 
table_Educ.schema
client.list_rows(table_Educ, max_results=5).to_dataframe()

# Keep All the Data in a DataFrame 
#allData =  client.list_rows(table_Educ).to_dataframe()

country_series_definitions
country_summary
international_education
series_summary
C:\Users\Lefteris\AppData\Local\Programs\Python\Python36\lib\site-packages\google\cloud\bigquery\table.py:1860: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  if not self._validate_bqstorage(bqstorage_client, create_bqstorage_client):


,country_name,country_code,indicator_name,indicator_code,value,year
0,Dominica,DMA,Duration of compulsory education (years),SE.COM.DURS,1.200000e+01,2016
1,Dominica,DMA,Population of the official entrance age to sec...,UIS.SAP.23.GPV.G1.M,4.790000e+02,2016
2,Equatorial Guinea,GNQ,GDP per capita (current US$),NY.GDP.PCAP.CD,8.747353e+03,2016
3,Equatorial Guinea,GNQ,"Population, ages 15-64, male",SP.POP.1564.MA.IN,4.268670e+05,2016
4,Equatorial Guinea,GNQ,"Population, total",SP.POP.TOTL,1.221490e+06,2016


___________________________________
#### Submitting a QUERY Tests
___________________________________

* Select country_name & country_code, as well as the Average(value)
* From the specific Dataset (international_education) with the specified Indicator Code 
* After year=2000 
* Group by the country_name & country_code in DESCENDING order 


In [20]:
query1 = """
SELECT country_name, country_code, AVG(value) AS average
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "SE.XPD.TOTL.GB.ZS"
  AND year > 2000
GROUP BY
  country_name,country_code
ORDER BY
  average DESC
;
        """

client = bigquery.Client()
query_job = client.query(query1)
response1 = query_job.to_dataframe()
response1

,country_name,country_code,average
0,Vanuatu,VUT,29.330663
1,Lesotho,LSO,27.614941
2,Tunisia,TUN,25.615756
3,Ethiopia,ETH,24.280294
4,Botswana,BWA,24.269573
...,...,...,...
169,Zambia,ZMB,7.548607
170,Lebanon,LBN,7.015822
171,Monaco,MCO,6.844026
172,Eritrea,ERI,5.601227


* Find the indicator_code & indicator_name
* From the specific Dataset (international_education) 
* For year 2016
* Group By indicator_code & indicator_name
* Which have mode than 175 records (countries) in DESCENDING order

In [21]:
# Your code goes here
code_count_query = """
SELECT 
    indicator_code, indicator_name, COUNT(1) AS num_rows 
FROM 
    `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
    year=2016
GROUP BY 
    indicator_code, indicator_name
HAVING COUNT(1) >= 175
ORDER BY COUNT(1) DESC"""

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(code_count_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
code_count_results = code_count_query_job.to_dataframe()

# View top few rows of results
print(code_count_results.head())



      indicator_code                       indicator_name  num_rows
0        SP.POP.GROW         Population growth (annual %)       232
1        SP.POP.TOTL                    Population, total       232
2     IT.NET.USER.P2      Internet users (per 100 people)       223
3  SP.POP.1564.TO.ZS  Population, ages 15-64 (% of total)       213
4  SP.POP.1564.MA.IN         Population, ages 15-64, male       213


----------------------------------------------
## ALL QUERIES NEEDED TO KEEP THE NEEDED DATA 
______________________________________________

_____________________
### DataFrame Names:
_____________________
#### Government expenditure on education as % of GDP (%)
> * AvgExpInEducOfGDP  
*indicator_code = "SE.XPD.TOTL.GB.ZS"*

#### GDP
> * AvgGDP
*indicator_code = "NY.GDP.MKTP.CD"

#### Economic Policy & Debt: Purchasing power parity
> * PurchasingPowerParity
*indicator_code = "NY.GDP.MKTP.PP.CD",

#### GDP per capita (constant 2010 US)
> * GDPperCapita
*indicator_code = "NY.GDP.PCAP.CD",

#### GDP per capita, PPP (current international Dollars)
> * GDPperCapitaIntern
*indicator_code = "NY.GDP.PCAP.PP.CD",

#### Internet users (per 100 people)
> * InternetUsers
*indicator_code = "IT.NET.USER.P2",

#### Population growth (annual%)
> * PopGrowth
*indicator_code = "SP.POP.GROW",

#### Total Population
> * TotalPop
*indicator_code = "SP.POP.TOTL"
  

In [22]:
#Government expenditure on education as % of GDP (%)
query1 = """
SELECT country_name, country_code, AVG(value) AS Average
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "SE.XPD.TOTL.GB.ZS"
  AND year > 2000
GROUP BY
  country_name,country_code
ORDER BY
  average DESC
;
        """
#safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(query1)# , job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
AvgExpInEducOfGDP = code_count_query_job.to_dataframe()
AvgExpInEducOfGDP = AvgExpInEducOfGDP.rename(columns={"Average": "SE.XPD.TOTL.GB.ZS"})

# View top few rows of results
print(AvgExpInEducOfGDP.head())

  country_name country_code  SE.XPD.TOTL.GB.ZS
0      Vanuatu          VUT          29.330663
1      Lesotho          LSO          27.614941
2      Tunisia          TUN          25.615756
3     Ethiopia          ETH          24.280294
4     Botswana          BWA          24.269573


In [23]:
# NY.GDP.MKTP.CD # GDP -> Use Avg for all years 
queryGDP = """
SELECT country_name, country_code, AVG(value) as avg2 
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "NY.GDP.MKTP.CD"
  AND year > 2000
GROUP BY country_name,country_code ;
        """
#safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(queryGDP)# , job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
AvgGDP = code_count_query_job.to_dataframe()
AvgGDP = AvgGDP.rename(columns={"avg2": "NY.GDP.MKTP.CD"})
AvgGDP = AvgGDP.drop(AvgGDP.columns[[0]], axis=1)

# View top few rows of results
print(AvgGDP.head())

  country_code  NY.GDP.MKTP.CD
0          CPV    1.387443e+09
1          OMN    5.031472e+10
2          SMR    1.900725e+09
3          SGP    2.023688e+11
4          CAN    1.389026e+12


In [24]:
# NY.GDP.MKTP.PP.CD # Economic Policy & Debt: Purchasing power parity
queryGDP_MKTP = """
SELECT country_name, country_code, AVG(value) as avg2 
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "NY.GDP.MKTP.PP.CD"
  AND year > 2000
GROUP BY country_name,country_code ;
        """
#safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(queryGDP_MKTP)# , job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
PurchasingPowerParity = code_count_query_job.to_dataframe()
PurchasingPowerParity = PurchasingPowerParity.rename(columns={"avg2": "NY.GDP.MKTP.PP.CD"})
PurchasingPowerParity = PurchasingPowerParity.drop(PurchasingPowerParity.columns[[0]], axis=1)

# View top few rows of results
print(PurchasingPowerParity.head())

  country_code  NY.GDP.MKTP.PP.CD
0          TUV       3.138429e+07
1          SSA       2.557237e+12
2          RUS       2.585010e+12
3          LCA       1.864379e+09
4          EAS       2.412017e+13


In [25]:
# NY.GDP.PCAP.CD # GDP per capita (constant 2010 US$)
queryGDP_PCAP = """
SELECT country_name, country_code, AVG(value) as avg2 
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "NY.GDP.PCAP.CD"
  AND year > 2000
GROUP BY country_name,country_code ;
        """
#safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(queryGDP_PCAP)# , job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
GDPperCapita = code_count_query_job.to_dataframe()
GDPperCapita = GDPperCapita.rename(columns={"avg2": "NY.GDP.PCAP.CD"})
GDPperCapita = GDPperCapita.drop(GDPperCapita.columns[[0]], axis=1)

# View top few rows of results
print(GDPperCapita.head())

  country_code  NY.GDP.PCAP.CD
0          MDG      372.726487
1          YEM     1069.195282
2          STP     1110.749813
3          SLB     1320.324367
4          SOM      421.412978


In [26]:
#NY.GDP.PCAP.PP.CD # GDP per capita, PPP (current international $)

queryGDP_PCAP_PP = """
SELECT country_name, country_code, AVG(value) as avg2 
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "NY.GDP.PCAP.PP.CD"
  AND year > 2000
GROUP BY country_name,country_code ;
        """
#safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(queryGDP_PCAP_PP)# , job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
GDPperCapitaIntern = code_count_query_job.to_dataframe()
GDPperCapitaIntern = GDPperCapitaIntern.rename(columns={"avg2": "NY.GDP.PCAP.PP.CD"})
GDPperCapitaIntern = GDPperCapitaIntern.drop(GDPperCapitaIntern.columns[[0]], axis=1)

# View top few rows of results
print(GDPperCapitaIntern.head())

  country_code  NY.GDP.PCAP.PP.CD
0          PAK        4025.856469
1          IRQ       12129.163562
2          SYC       20228.458891
3          CAF         756.868040
4          YEM        3646.242972


In [27]:
# IT.NET.USER.P2 #Internet users (per 100 people)

queryNET_USER = """
SELECT country_name, country_code, AVG(value) as avg2 
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "IT.NET.USER.P2"
  AND year > 2000
GROUP BY country_name,country_code ;
        """
#safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(queryNET_USER)# , job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
InternetUsers = code_count_query_job.to_dataframe()
InternetUsers = InternetUsers.rename(columns={"avg2": "IT.NET.USER.P2"})
InternetUsers = InternetUsers.drop(InternetUsers.columns[[0]], axis=1)

# View top few rows of results
print(InternetUsers.head())

  country_code  IT.NET.USER.P2
0          KNA       48.299515
1          CAF        1.646076
2          BWA       13.319948
3          MHL        8.837881
4          UZB       15.898729


In [28]:
# SP.POP.GROW # Population growth (annual %)

queryPOP_GROW = """
SELECT country_name, country_code, AVG(value) as avg2 
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "SP.POP.GROW"
  AND year > 2000
GROUP BY country_name,country_code ;
        """
#safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(queryPOP_GROW)# , job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
PopGrowth = code_count_query_job.to_dataframe()
PopGrowth = PopGrowth.rename(columns={"avg2": "SP.POP.GROW"})
PopGrowth = PopGrowth.drop(PopGrowth.columns[[0]], axis=1)

# View top few rows of results
print(PopGrowth.head())

  country_code  SP.POP.GROW
0          TKM     1.413860
1          GAB     2.969143
2          BRN     1.493559
3          AGO     3.506682
4          MIC     1.198141


In [29]:
# SP.POP.TOTL # Total Population 

queryPOP_TOTL = """
SELECT country_name, country_code, AVG(value) as avg2 
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "SP.POP.TOTL"
  AND year > 2000
GROUP BY country_name,country_code ;
        """
#safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(queryPOP_TOTL)# , job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
TotalPop = code_count_query_job.to_dataframe()
TotalPop = TotalPop.rename(columns={"avg2": "SP.POP.TOTL"})
TotalPop = TotalPop.drop(TotalPop.columns[[0]], axis=1)

# View top few rows of results
print(TotalPop.head())

  country_code   SP.POP.TOTL
0          GNQ  9.047514e+05
1          LBN  4.465734e+06
2          ETH  8.475392e+07
3          LCN  5.854836e+08
4          BRB  2.778351e+05


----------------------------
#### Merge All DF to create the **WorldBankData**
----------------------------

In [30]:
import pandas as pd 
#final = pd.merge(AvgExpInEducOfGDP,AvgGDP, PurchasingPowerParity, GDPperCapita, GDPperCapitaIntern, InternetUsers, PopGrowth, TotalPop, on = 'country_code')
final = pd.merge(AvgExpInEducOfGDP,AvgGDP, on = 'country_code')
print(final.shape)

final = pd.merge(final,PurchasingPowerParity, on = 'country_code')
print(final.shape)

final = pd.merge(final,GDPperCapita, on = 'country_code')
print(final.shape)

final = pd.merge(final,GDPperCapitaIntern, on = 'country_code')
print(final.shape)

final = pd.merge(final,InternetUsers, on = 'country_code')
print(final.shape)

final = pd.merge(final,PopGrowth, on = 'country_code')
print(final.shape)

WorldBankData = pd.merge(final,TotalPop, on = 'country_code')
print(final.shape)
WorldBankData = WorldBankData.rename(columns={"country_name": "country","country_code": "iso_code" })
WorldBankData = WorldBankData.drop(WorldBankData.columns[[0]], axis=1)

# FINAL DATAFRAME
print(WorldBankData.head())

(174, 4)
(172, 5)
(172, 6)
(172, 7)
(171, 8)
(171, 9)
(171, 9)
  iso_code  SE.XPD.TOTL.GB.ZS  NY.GDP.MKTP.CD  NY.GDP.MKTP.PP.CD  \
0      VUT          29.330663    5.741323e+08       6.174113e+08   
1      LSO          27.614941    1.943798e+09       4.477110e+09   
2      TUN          25.615756    3.822997e+10       9.825796e+10   
3      ETH          24.280294    3.043921e+10       8.823511e+10   
4      BWA          24.269573    1.149685e+10       2.556843e+10   

   NY.GDP.PCAP.CD  NY.GDP.PCAP.PP.CD  IT.NET.USER.P2  SP.POP.GROW  \
0     2445.929832        2662.082984        9.483872     2.370084   
1      949.537293        2187.561326        8.223656     1.030939   
2     3611.317354        9261.094797       27.038320     1.011595   
3      336.265184         992.715519        2.892368     2.694717   
4     5717.157297       12706.184367       13.319948     1.649277   

    SP.POP.TOTL  
0  2.286927e+05  
1  2.024561e+06  
2  1.051604e+07  
3  8.475392e+07  
4  1.978834e+06  


In [31]:
Final_DF = pd.merge(WorldBankData,df5, on = 'iso_code')
print(Final_DF.shape)
Final_DF.to_csv("outputs/complete_df.csv")
Final_DF.head(10)

(117, 34)


,iso_code,SE.XPD.TOTL.GB.ZS,NY.GDP.MKTP.CD,NY.GDP.MKTP.PP.CD,NY.GDP.PCAP.CD,NY.GDP.PCAP.PP.CD,IT.NET.USER.P2,SP.POP.GROW,SP.POP.TOTL,country,...,april_total_vaccinations,may_total_vaccinations,january_daily_vaccinations,february_daily_vaccinations,march_daily_vaccinations,april_daily_vaccinations,may_daily_vaccinations,religiousity%,population,urban_population_percent
0,TUN,25.615756,3.822997e+10,9.825796e+10,3611.317354,9261.094797,27.038320,1.011595,1.051604e+07,Tunisia,...,3.142615,3.754165,0.000000,0.000000,0.021419,0.083333,0.109492,93,11839918,70
1,ETH,24.280294,3.043921e+10,8.823511e+10,336.265184,992.715519,2.892368,2.694717,8.475392e+07,Ethiopia,...,0.965483,1.053355,0.000000,0.000000,0.000000,0.026442,0.020248,99,115434444,21
2,BWA,24.269573,1.149685e+10,2.556843e+10,5717.157297,12706.184367,13.319948,1.649277,1.978834e+06,Botswana,...,2.114016,0.000000,0.000000,0.000000,0.058775,0.062022,0.000000,77,2359585,73
3,ZWE,24.147020,9.683205e+09,2.371541e+10,676.168937,1693.316229,7.851712,1.741777,1.391370e+07,Zimbabwe,...,3.358052,4.076271,0.000000,0.006989,0.014450,0.080060,0.124980,88,14899771,38
4,GHA,23.838494,2.617094e+10,7.219429e+10,1052.578863,2954.056455,9.755382,2.489686,2.367518e+07,Ghana,...,2.724465,0.000000,0.000000,0.000000,0.063624,0.034104,0.000000,95,31181428,57
5,NAM,22.968228,9.032565e+09,1.725670e+10,4110.474397,7855.359347,9.955710,1.666750,2.159267e+06,Namibia,...,0.784921,0.000000,0.000000,0.000000,0.002508,0.017980,0.000000,92,2548663,55
6,DJI,22.524957,1.021112e+09,2.022896e+09,1210.807621,2412.492157,4.960999,1.702929,8.333628e+05,Djibouti,...,1.034482,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,98,990447,79
7,KEN,22.053427,3.686106e+10,9.583315e+10,879.088032,2342.190722,8.324680,2.702229,3.995911e+07,Kenya,...,1.580695,0.000000,0.000000,0.000000,0.008013,0.045663,0.000000,94,53968739,28
8,CRI,21.683806,3.358104e+10,5.456582e+10,7389.975526,12114.256016,35.045882,1.331239,4.446616e+06,Costa Rica,...,16.049758,18.624511,0.032383,0.048854,0.163381,0.299137,0.363500,79,5102158,80
9,GTM,21.366231,3.989220e+10,9.212244e+10,2729.653234,6393.669073,12.318379,2.206101,1.419701e+07,Guatemala,...,0.950584,1.014569,0.000000,0.004172,0.017419,0.013089,0.006419,88,17971382,52
